&emsp;&emsp;自然语言处理中的一个常见任务是分类。该任务的目标是训练一个模型，为一些输入文本分配一个标签或类(见下图)。从情感分析和意图检测到提取实体和检测语言，对文本进行分类在全球范围内被广泛应用。无论是代表性的还是生成性的语言模型，对分类的影响都不容小觑。

<div style="text-align: center;">  
    <img src="https://bohrium.oss-cn-zhangjiakou.aliyuncs.com/article/183329/a82672a606434fbbb376b733987dfd5f/2b8b7ae2-df4b-4a38-9236-71d3a6049ab8.png" alt="alt text">  
</div>

&emsp;&emsp;在本章中，我们将讨论几种使用语言模型对文本进行分类的方法。它将作为使用已经训练过的语言模型的一个容易理解的介绍。由于文本分类的领域很广，我们将讨论几种技术，并使用它们来探索语言模型领域:

- “使用表示模型的文本分类”：展示了非生成模型用于分类的灵活性。我们将涵盖特定于任务的模型和嵌入模型。
- “基于生成模型的文本分类”：是对生成语言模型的介绍，因为大多数生成语言模型都可以用于分类。我们将涵盖开源和闭源语言模型。

&emsp;&emsp;在本章中，我们将重点关注利用预训练的语言模型，这些模型已经在大量数据上进行了训练，可以用于对文本进行分类。如下图所示，我们将研究表示模型和语言模型，并探讨它们之间的差异。

<div style="text-align: center;">  
    <img src="https://bohrium.oss-cn-zhangjiakou.aliyuncs.com/article/183329/62aee1f83c03400e8392274400fb525c/366477f9-2392-4dc3-910a-9e737527b60a.png">  
</div>

本章作为各种语言模型的介绍，包括生成和非生成。我们将会遇到加载和使用这些模型的常用包。

# 电影评论的情感

&emsp;&emsp;我们将使用Hugging Face中著名的“rotten_tomatoes”数据集来训练和评估我们的模型。它包含了来自烂番茄的5331条正面和5331条负面的电影评论

In [ ]:
from datasets import load_dataset

# Load our data
data = load_dataset("rotten_tomatoes")
print(data)
data["train"][0, -1]

由此，可以得到以下结果：

In [ ]:
DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 8530
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 1066
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 1066
    })
})
{'text': ['the rock is destined to be the 21st century\'s new " conan " and that he\'s going to make a splash even greater than arnold schwarzenegger , jean-claud van damme or steven segal .',
  'things really get weird , though not particularly scary : the movie is all portent and no content .'],
 'label': [1, 0]}

&emsp;&emsp;从第1~14行运行结果来看，数据被分成训练、测试和验证三部分。在本章中，我们将在训练模型时使用训练分割，在验证结果时使用测试分割。请注意，如果使用训练分割和测试分割来执行超参数调优，则可以使用额外的验证分割来进一步验证泛化。

&emsp;&emsp;从15~17行运行结果来看，这些简短的评论要么被标记为正面(1)，要么被标记为负面(0)，这意味着我们将专注于二元情感分类。

# 使用表示模型进行文本分类

&emsp;&emsp;使用预训练的表示模型进行分类通常有两种方式，要么使用特定于任务的模型，要么使用嵌入模型。正如我们在前一章中所探讨的，这些模型是通过在特定的下游任务上微调基础模型(如BERT)来创建的，如下图所示。

<div style="text-align: center;">  
    <img src="https://bohrium.oss-cn-zhangjiakou.aliyuncs.com/article/183329/4888a18216c342838194f73bfaca0007/d7873309-86e7-4028-9e8d-5b8e1bd640ab.png">  
</div>


&emsp;&emsp;特定于任务的模型是一种表示模型，如BERT，为特定任务(如情感分析)训练。正如我们在第1章中探讨的那样，嵌入模型生成通用的嵌入，可以用于各种不局限于分类的任务，比如语义搜索(参见第8章)。

&emsp;&emsp;微调用于分类的BERT模型的过程将在第11章中介绍，而创建嵌入模型将在第10章中介绍。在本章中，我们将两个模型保持冻结状态(不可训练)，并且只使用它们的输出，如图4-4所示。

<div style="text-align: center;">  
    <img src="https://bohrium.oss-cn-zhangjiakou.aliyuncs.com/article/183329/4888a18216c342838194f73bfaca0007/d32ccdab-1b01-4874-a768-7e8b897801bb.png">  
</div>

&emsp;&emsp;我们将利用其他人已经为我们进行微调的预训练模型，并探索如何使用它们对我们选择的电影评论进行分类。

# 模型选择

&emsp;&emsp;在Hugging Face Hub上有超过6万个用于文本分类的模型，在写作的那一刻有超过8000个模型生成嵌入。此外，选择一个适合你用例的模型，并考虑它的语言兼容性、底层架构、大小和性能，这一点至关重要。

&emsp;&emsp;让我们从底层架构开始。正如我们在第1章中所探讨的，BERT，一个众所周知的仅编码的架构，是创建特定任务和嵌入模型的流行选择。虽然生成模型(如GPT家族)是令人难以置信的模型，但仅编码器模型同样在特定任务的用例中表现出色，并且往往在尺寸上明显更小。

&emsp;&emsp;多年来，已经开发了许多BERT的变体，包括RoBERTa，DistilBERT，ALBERT和DeBERTa，每个都在不同的环境中训练。我们可以在下图中找到一些著名的类BERT模型的概述。

<div style="text-align: center;">  
    <img src="https://bohrium.oss-cn-zhangjiakou.aliyuncs.com/article/183329/2414ecba2a124e8b91ee67203d8a8b28/bc4dc45a-ebc2-4b68-bb56-b60fd262550e.png"> 
</div>

&emsp;&emsp;选择一个合适的模型本身就是一门艺术。尝试在 Hugging Face 的 Hub 上找到的数千个预训练模型是不可行的，所以我们需要高效地使用我们选择的模型。话虽如此，有几个模型是很好的起点，可以让我们对这类模型的基本性能有一个了解。把它们看作是坚实的基准:

- BERT base model (uncased)

- RoBERTa base model

- DistilBERT base model (uncased)

- DeBERTa base model

- bert-tiny

- ALBERT base v2

&emsp;&emsp;对于特定于任务的模型，我们选择Twitter-RoBERTa-base for Sentiment Analysis（基于Twitter - RoBERTa的情感分析）模型。这是一个RoBERTa模型，对推文进行了微调，用于情感分析。虽然这不是专门为电影评论训练的，但探索这个模型如何泛化是很有趣的。

&emsp;&emsp;在选择要从中生成嵌入的模型时，[MTEB排行榜](https://huggingface.co/spaces/mteb/leaderboard)是一个很好的起点。它包含开放和闭源模型，对多个任务（模型大小、使用内存、最大词元数、平均性能等）进行基准测试。确保不仅只考虑性能。在现实解决方案中，推理速度的重要性不容低估。因此，我们将在本节中使用sentence-transformer /all-mpnet-base-v2作为嵌入。它是一个小而高性能的模型。

# 使用特定任务模型

&emsp;&emsp;现在我们已经选择了我们的任务特定的表示模型，让我们开始加载我们的模型:

In [ ]:
from transformers import pipeline

# Path to our HF model
model_path = "cardiffnlp/twitter-roberta-base-sentiment-latest"

# Load model into pipeline
pipe = pipeline(
    model=model_path,
    tokenizer=model_path,
    return_all_scores=True,
    device="cuda:0"
)

&emsp;&emsp;当我们加载我们的模型时，我们也加载了tokenizer，它负责将输入文本转换为单个token，如下图所示。虽然这个参数是不需要的，因为它是自动加载的，但它说明了幕后发生的事情。
<div style="text-align: center;">  
    <img src="https://bohrium.oss-cn-zhangjiakou.aliyuncs.com/article/183329/c54a690183a2446f8463783c04bd314d/7a73873a-a694-4faf-a3bc-8f9508b66d3e.png"> 
</div>

&emsp;&emsp;这些标记是大多数语言模型的核心，第2章将深入探讨。这些标记的一个主要好处是，即使它们不在训练数据中，也可以将它们组合起来生成表示，如下图所示。

<div style="text-align: center;">  
    <img src="https://bohrium.oss-cn-zhangjiakou.aliyuncs.com/article/183329/c54a690183a2446f8463783c04bd314d/70676728-4b15-4e23-ad6a-0e7dd981fdb3.png"> 
</div>

&emsp;&emsp;在加载了所有必要的组件之后，我们可以继续在我们的数据的测试分割上使用我们的模型，先预测后评估：

In [ ]:
import numpy as np
from tqdm import tqdm
from transformers.pipelines.pt_utils import KeyDataset

# Run inference
y_pred = []
for output in tqdm(pipe(KeyDataset(data["test"], "text")), total=len(data["test"])):
    negative_score = output[0]["score"]
    positive_score = output[2]["score"]
    assignment = np.argmax([negative_score, positive_score])
    y_pred.append(assignment)

from sklearn.metrics import classification_report

def evaluate_performance(y_true, y_pred):
    """Create and print the classification report"""
    performance = classification_report(
        y_true, y_pred,
        target_names=["Negative Review", "Positive Review"]
    )
    print(performance)
    
evaluate_performance(data["test"]["label"], y_pred)

生成的分类报告如下：

In [ ]:
                    precision  recall   f1-score   support

Negative Review       0.76      0.88      0.81       533
Positive Review       0.86      0.72      0.78       533

       accuracy                           0.80      1066
      macro avg       0.81      0.80      0.80      1066
   weighted avg       0.81      0.80      0.80      1066

&emsp;&emsp;要阅读生成的分类报告，我们首先要探索如何识别正确和不正确的预测。根据我们对某件事的预测是正确(True)还是不正确(False)，以及我们预测的类别是正确(Positive)还是不正确(Negative)，有四种组合。我们可以用一个矩阵来说明这些组合，通常称为混淆矩阵，如下图所示。

<div style="text-align: center;">  
    <img src="https://bohrium.oss-cn-zhangjiakou.aliyuncs.com/article/183329/a70db392760b470e8487433b3600fcf4/f44b46f7-52f6-4f2b-b6cf-7927b56ee856.png"> 
</div>

&emsp;&emsp;使用混淆矩阵，我们可以推导出几个公式来描述模型的质量。在之前生成的分类报告中，我们可以看到四种这样的方法，分别是precision、recall、accuracy和F1分数:

- 精确率衡量的是找到的项目中有多少是相关的，这表明了相关结果的准确性。

- 召回率是指找到了多少个相关类，这表明它有能力找到所有相关的结果。

- 准确性是指模型在所有预测中做出了多少正确的预测，这表明模型的整体正确性。

- F1分数平衡了准确率和召回率，从而创造了模型的整体表现。

&emsp;&emsp;这四个指标如下图所示，下图使用前面提到的分类报告对它们进行了描述：

<div style="text-align: center;">  
    <img src="https://bohrium.oss-cn-zhangjiakou.aliyuncs.com/article/183329/a70db392760b470e8487433b3600fcf4/396a1f5b-ee3e-4b55-b02c-7356275bbb97.png"> 
</div>

&emsp;&emsp;我们将在本书的所有例子中考虑F1分数的加权平均值，以确保每个班级都得到平等对待。我们预训练的BERT模型给了我们一个0.80的F1分数(我们是从加权的avg行和F1 -score列中读取的)，这对于一个没有专门在我们的领域数据上训练的模型来说是非常棒的!

&emsp;&emsp;为了提高我们选择的模型的性能，我们可以做一些不同的事情，包括选择一个在我们的领域数据上训练过的模型，在这种情况下是电影评论，比如[DistilBERT base uncased finetuned SST-2](https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english)。我们还可以将我们的注意力转移到另一种风格的表示模型上，即嵌入模型。

# 利用嵌入的分类任务
&emsp;&emsp;在前面的例子中，我们使用了一个预训练的任务特定模型来进行情感分析。但是，如果我们找不到针对这个特定任务进行预训练的模型怎么办?我们需要自己对表征模型进行微调吗?答案是否定的!

&emsp;&emsp;如果你有足够的计算能力，有时你可能想要自己对模型进行微调(参见第11章)。然而，并不是每个人都有机会使用广泛的计算。这就是通用嵌入模型的用武之地。

## 监督分类
&emsp;&emsp;与前面的例子不同，我们可以通过从更经典的角度接近它来自己执行部分训练过程。与直接使用表示模型进行分类不同，我们将使用嵌入模型来生成特征。然后可以将这些特征馈送到分类器中，从而创建如下图所示的两步方法。

<div style="text-align: center;">  
    <img src="https://bohrium.oss-cn-zhangjiakou.aliyuncs.com/article/183329/a70db392760b470e8487433b3600fcf4/b63f8381-fc89-4dee-927d-0ee64e55d7af.png"> 
</div>



&emsp;&emsp;这种分离的一个主要好处是，微调模型比较昂贵，而我们不需要微调我们的嵌入模型。相反，我们可以在CPU上训练一个分类器，比如逻辑回归。

&emsp;&emsp;在第一步中，我们使用如下图所示的嵌入模型将文本输入转换为嵌入。请注意，这个模型同样保持冻结状态，在训练过程中不会更新。

<div style="text-align: center;">  
    <img src="https://bohrium.oss-cn-zhangjiakou.aliyuncs.com/article/183329/a70db392760b470e8487433b3600fcf4/786b3495-54eb-4e90-8268-cf24d909f658.png"> 
</div>

我们可以用 sentence-transformer，这是一个利用预训练嵌入模型的流行包。创建嵌入很简单:

In [ ]:
 from sentence_transformers import SentenceTransformer

# Load model
model = SentenceTransformer("sentence-transformers/all-mpnet-base-v2")

# Convert text to embeddings
train_embeddings = model.encode(data["train"]["text"], show_progress_bar=True)
test_embeddings = model.encode(data["test"]["text"], show_progress_bar=True)

&emsp;&emsp;这些嵌入是输入文本的数字表示。嵌入的值的数量，或维度，取决于底层的嵌入模型。该模型的大小为：

In [ ]:
train_embeddings.shape

#输出为(8530, 768)

&emsp;&emsp;这表明，我们的8,530个输入文档中的每一个都有一个768的嵌入维数，因此每个嵌入包含768个数值。

&emsp;&emsp;在第二步中，这些嵌入作为分类器的输入特征，如下图所示。分类器是可训练的，不局限于逻辑回归，只要它执行分类，就可以采取任何形式。

<div style="text-align: center;">  
    <img src="https://bohrium.oss-cn-zhangjiakou.aliyuncs.com/article/183329/4a539b463f1d460ba5b00663afff676b/c5a95f9c-e2ef-4499-ba11-feb887be3362.png"> 
</div>

&emsp;&emsp;我们将保持这一步的简单性，并使用逻辑回归作为分类器。为了训练它，我们只需要将生成的嵌入与我们的标签一起使用，并对它进行评估:


In [ ]:
from sklearn.linear_model import LogisticRegression

# Train a logistic regression on our train embeddings
clf = LogisticRegression(random_state=42)
clf.fit(train_embeddings, data["train"]["label"])

# Predict previously unseen instances
y_pred = clf.predict(test_embeddings)
evaluate_performance(data["test"]["label"], y_pred)

评估结果为：

In [ ]:
                   precision   recall   f1-score   support

Negative Review       0.85      0.86      0.85       533
Positive Review       0.86      0.85      0.85       533

       accuracy                           0.85      1066
      macro avg       0.85      0.85      0.85      1066
   weighted avg       0.85      0.85      0.85      1066

&emsp;&emsp;通过在我们的嵌入之上训练一个分类器，我们成功地获得了0.85的F1分数!这证明了在保持底层嵌入模型冻结的情况下训练轻量级分类器的可能性

## 如果我们没有标记数据怎么办?

&emsp;&emsp;在我们之前的例子中，我们已经标记了我们可以利用的数据，但在实践中可能并不总是如此。获得标记数据是一项资源密集型任务，可能需要大量的人力劳动。此外，收集这些标签真的值得吗?

&emsp;&emsp;为了测试这一点，我们可以执行零射击分类，我们没有标记的数据来探索任务是否可行。虽然我们知道标签的定义(它们的名字)，但我们没有标记的数据来支持它们。零射击分类尝试预测输入文本的标签，即使它没有对它们进行训练，如下图所示。

<div style="text-align: center;">  
    <img src="https://bohrium.oss-cn-zhangjiakou.aliyuncs.com/article/183329/2b00ab9d28474da2bafb1f2451ecee05/24987d89-2451-426f-a853-245231e4f10d.png"> 
</div>

&emsp;&emsp;要用嵌入来执行零射击分类，我们可以使用一个简洁的技巧。我们可以根据标签应该代表什么来描述我们的标签。例如，电影评论的负面标签可以描述为“这是一篇负面的电影评论”。通过描述和嵌入标签和文档，我们就有了可以处理的数据。如下图所示，这个过程允许我们生成自己的目标标签，而不需要实际拥有任何标记的数据。

<div style="text-align: center;">  
    <img src="https://bohrium.oss-cn-zhangjiakou.aliyuncs.com/article/183329/2b00ab9d28474da2bafb1f2451ecee05/0af761e4-5cbb-4f0e-b68e-5eb19d417271.png"> 
</div>

&emsp;&emsp;我们可以创建这些标签嵌入 .encode 函数，就像我们之前做的那样:

In [ ]:
# Create embeddings for our labels
label_embeddings = model.encode(["A negative review",  "A positive review"])

&emsp;&emsp;为了给文档分配标签，我们可以对文档标签对应用余弦相似度。这是向量之间夹角的余弦，通过嵌入的点积计算，并除以它们长度的乘积，如下图所示。

<div style="text-align: center;">  
    <img src="https://bohrium.oss-cn-zhangjiakou.aliyuncs.com/article/183329/2b00ab9d28474da2bafb1f2451ecee05/25272811-de2e-405a-beaa-631ffaf7f8b8.png"> 
</div>

&emsp;&emsp;我们可以使用余弦相似度来检查给定文档与候选标签的描述有多相似。选择与文档相似度最高的标签，如下图所示。

<div style="text-align: center;">  
    <img src="https://bohrium.oss-cn-zhangjiakou.aliyuncs.com/article/183329/2b00ab9d28474da2bafb1f2451ecee05/080a5083-5917-405a-89cb-b13d7f6da0a7.png"> 
</div>

&emsp;&emsp;为了在嵌入上执行余弦相似度，我们只需要将文档嵌入与标签嵌入进行比较，得到最佳匹配对:

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

# Find the best matching label for each document
sim_matrix = cosine_similarity(test_embeddings, label_embeddings)
y_pred = np.argmax(sim_matrix, axis=1)

evaluate_performance(data["test"]["label"], y_pred)

&emsp;&emsp;我们只需要为我们的标签命名来执行我们的分类任务。让我们看看这个方法是如何工作的:

In [ ]:
from datasets import load_dataset

# Load our data
data = load_dataset("rotten_tomatoes")

from transformers import pipeline

# Path to our HF model
model_path = "cardiffnlp/twitter-roberta-base-sentiment-latest"

# Load model into pipeline
pipe = pipeline(
    model=model_path,
    tokenizer=model_path,
    return_all_scores=True,
    device="cuda:0"
)

import numpy as np
from tqdm import tqdm
from transformers.pipelines.pt_utils import KeyDataset
from sentence_transformers import SentenceTransformer

# Load model
model = SentenceTransformer("sentence-transformers/all-mpnet-base-v2")

# Convert text to embeddings
train_embeddings = model.encode(data["train"]["text"], show_progress_bar=True)
test_embeddings = model.encode(data["test"]["text"], show_progress_bar=True)

# Run inference
y_pred = []
for output in tqdm(pipe(KeyDataset(data["test"], "text")), total=len(data["test"])):
    negative_score = output[0]["score"]
    positive_score = output[2]["score"]
    assignment = np.argmax([negative_score, positive_score])
    y_pred.append(assignment)

from sklearn.metrics import classification_report

def evaluate_performance(y_true, y_pred):
    """Create and print the classification report"""
    performance = classification_report(
        y_true, y_pred,
        target_names=["Negative Review", "Positive Review"]
    )
    print(performance)
    

# Create embeddings for our labels
label_embeddings = model.encode(["A negative review",  "A positive review"])

from sklearn.metrics.pairwise import cosine_similarity

# Find the best matching label for each document
sim_matrix = cosine_similarity(test_embeddings, label_embeddings)
y_pred = np.argmax(sim_matrix, axis=1)

evaluate_performance(data["test"]["label"], y_pred)

In [ ]:
                   precision   recall   f1-score   support

Negative Review       0.78      0.77      0.78       533
Positive Review       0.77      0.79      0.78       533

       accuracy                           0.78      1066
      macro avg       0.78      0.78      0.78      1066
   weighted avg       0.78      0.78      0.78      1066


&emsp;&emsp;考虑到我们根本没有使用任何标记数据，0.78的F1分数是相当令人印象深刻的!这正好说明了嵌入是多么的通用和有用，特别是如果你对它们的使用方式有点创意的话。

<div style="border: 1px solid #000; padding: 10px; background-color: #f9f9f9;">  
    <div style="text-align: center;"> 
    <p style="font-weight: bold;">提示：</p>  
    </div>
    <p>&emsp;&emsp;我们决定用“差评/好评”作为我们的标签名称，但这还可以改进。相反，我们可以通过使用“非常负面/正面的电影评论”来代替，使它们对我们的数据更具体和具体。这样，嵌入将捕捉到它是一篇电影评论，并将更多地关注两个标签的极端情况。试一试，探索一下它是如何影响结果的。</p>  
</div>

# 使用生成模型进行文本分类

&emsp;&emsp;使用生成语言模型(如OpenAI的GPT模型)进行分类，其工作原理与我们迄今为止所做的略有不同。这些模型将一些文本和生成文本作为输入，因此被恰当地命名为序列到序列模型。这与我们的任务特定模型形成鲜明对比，我们的任务特定模型输出一个类，如下图所示。

<div style="text-align: center;">  
    <img src="https://bohrium.oss-cn-zhangjiakou.aliyuncs.com/article/183329/8f45a0bfc0814a3c981d834f18f5c171/4e7cf5c6-131f-4d3a-a77f-7a0f26a739a6.png"> 
</div>


&emsp;&emsp;这些生成模型通常是在各种各样的任务上训练的，通常不会开箱即用地执行你的用例。例如，如果我们在没有任何上下文的情况下给生成模型一个电影评论，它就不知道该怎么做。

&emsp;&emsp;相反，我们需要帮助它理解上下文，并引导它找到我们正在寻找的答案。如下图所示，这个引导过程主要是通过你给这样一个模型的指令或提示来完成的。迭代地改进你的提示以得到你喜欢的输出被称为提示工程。

<div style="text-align: center;">  
    <img src="https://bohrium.oss-cn-zhangjiakou.aliyuncs.com/article/183329/8f45a0bfc0814a3c981d834f18f5c171/1cba9a0c-5c03-4477-8103-e9dac380e5cf.png"> 
</div>

&emsp;&emsp;在本节中，我们将演示如何在没有我们的烂番茄数据集的情况下，利用不同类型的生成模型来执行分类。

## 使用Text-to-Text Transfer Transformer

&emsp;&emsp;在本书中，我们将主要探索像BERT这样的纯编码器(表示)模型和像ChatGPT这样的纯解码器(生成)模型。然而，正如第1章所讨论的，最初的Transformer架构实际上由一个编码器-解码器架构组成。与仅解码器模型一样，这些编码器-解码器模型是序列到序列的模型，一般属于生成模型的范畴。

&emsp;&emsp;利用这种体系结构的一个有趣的模型家族是文本到文本传输转换器或T5模型。如下图所示，它的架构类似于最初的Transformer, 12个解码器和12个编码器堆叠在一起。


<div style="text-align: center;">  
    <img src="https://bohrium.oss-cn-zhangjiakou.aliyuncs.com/article/183329/8f45a0bfc0814a3c981d834f18f5c171/2f455169-2f66-4b83-9207-fa03917a01c9.png"> 
</div>

&emsp;&emsp;在这种架构下，这些模型首先使用掩码语言建模进行预训练。在训练的第一步，如下图所示，在预训练期间，不是屏蔽单个词元，而是屏蔽一系列词元(或词元跨度)。

<div style="text-align: center;">  
    <img src="https://bohrium.oss-cn-zhangjiakou.aliyuncs.com/article/183329/8f45a0bfc0814a3c981d834f18f5c171/386a414e-b720-4c13-abf7-6e88cbc2a1fd.png"> 
</div>

&emsp;&emsp;训练的第二步，即对基础模型进行微调，才是真正神奇的地方。不是针对某一特定任务对模型进行微调，而是将每个任务转换为序列对序列的任务，并同时进行训练。如下图1所示，这使得模型可以在各种各样的任务上进行训练。

<div style="text-align: center;">  
    <img src="https://bohrium.oss-cn-zhangjiakou.aliyuncs.com/article/183329/8f45a0bfc0814a3c981d834f18f5c171/2bc849ee-f4b5-4a6d-b2e1-197a341e650d.png"> 
</div>



&emsp;&emsp;这种微调方法在论文“缩放指令-微调语言模型”中得到了扩展，该论文在微调过程中引入了一千多个任务，这些任务更紧密地遵循我们从GPT模型中了解到的指令。这就产生了Flan-T5系列模型，这些模型受益于各种各样的任务。

&emsp;&emsp;要使用这个预训练的Flan-T5模型进行分类，我们将首先通过 “text2text-generation” 任务，通常为这些编码器-解码器模型保留:

In [ ]:
# Load our model
pipe = pipeline(
    "text2text-generation", 
    model="google/flan-t5-small", 
    device="cuda:0"
)

&emsp;&emsp;Flan-T5型号有各种尺寸(Flan-T5 -small/base/large/xl/xxl)，我们会用最小的来加快一点速度。不过，你也可以随意摆弄大一点的型号，看看能不能改善效果。

&emsp;&emsp;与我们的任务特定模型相比，我们不能只是给模型一些文本，然后希望它能输出情绪。相反，我们必须指示模型这样做。

&emsp;&emsp;因此，我们在每个文档前加上提示“下面的句子是肯定的还是否定的?””:

In [ ]:
# Prepare our data
prompt = "Is the following sentence positive or negative? "
data = data.map(lambda example: {"t5": prompt + example['text']})
data

&emsp;&emsp;运行结果为


In [ ]:
DatasetDict({
    train: Dataset({
        features: ['text', 'label', 't5'],
        num_rows: 8530
    })
    validation: Dataset({
        features: ['text', 'label', 't5'],
        num_rows: 1066
    })
    test: Dataset({
        features: ['text', 'label', 't5'],
        num_rows: 1066
    })
})

&emsp;&emsp;在创建了我们更新的数据之后，我们可以运行类似于特定任务示例的pipeline：

In [ ]:
# Run inference
y_pred = []
for output in tqdm(pipe(KeyDataset(data["test"], "t5")), total=len(data["test"])):
    text = output[0]["generated_text"]
    y_pred.append(0 if text == "negative" else 1)

&emsp;&emsp;由于这个模型生成文本，我们确实需要将文本输出转换为数值。输出单词“negative”被映射为0，而“positive”被映射为1。

&emsp;&emsp;这些数值现在允许我们以与之前相同的方式测试模型的质量:

In [ ]:
evaluate_performance(data["test"]["label"], y_pred)

&emsp;&emsp;运行结果为：

In [ ]:
                   precision   recall   f1-score   support

Negative Review       0.83      0.85      0.84       533
Positive Review       0.85      0.83      0.84       533

       accuracy                           0.84      1066
      macro avg       0.84      0.84      0.84      1066
   weighted avg       0.84      0.84      0.84      1066

&emsp;&emsp;F1得分为0.84，很明显，这个Flan-T5模型是一个惊人的第一次看到生成模型的能力。

## ChatGPT for Classification

&emsp;&emsp;虽然我们在本书中专注于开源模型，但语言AI领域的另一个主要组成部分是闭源模型;特别是ChatGPT。

&emsp;&emsp;尽管原始ChatGPT模型(GPT-3.5)的底层体系结构不是共享的，但我们可以从它的名称中假设它基于目前为止我们在GPT模型中看到的仅解码器体系结构。

&emsp;&emsp;幸运的是，OpenAI分享了训练过程的概述，其中涉及一个重要的组件，即首选项调优。如下图所示，OpenAI首先手动创建所需的输出到输入提示符(指令数据)，并使用该数据创建其模型的第一个变体。

<div style="text-align: center;">  
    <img src="https://bohrium.oss-cn-zhangjiakou.aliyuncs.com/article/183329/86b3bf21d47045c983aa1b57637ee8ee/3a407e4b-d601-48b1-8b8c-a29c11b9a4a5.png"> 
</div>

&emsp;&emsp;OpenAI使用生成的模型来生成多个输出，这些输出被手动从最好到最差排序。如下图所示，该排序显示了对某些输出(偏好数据)的偏好，并用于创建其最终模型ChatGPT。

<div style="text-align: center;">  
    <img src="https://bohrium.oss-cn-zhangjiakou.aliyuncs.com/article/183329/86b3bf21d47045c983aa1b57637ee8ee/bc8a874b-52c5-4940-8a8e-2b2e21a7ecff.png"> 
</div>

&emsp;&emsp;与指令数据相比，使用偏好数据的一个主要好处是它所代表的细微差别。通过展示好的输出和更好的输出之间的差异，生成模型学习生成类似于人类偏好的文本。在第12章中，我们将探索这些微调和偏好调整方法是如何工作的，以及你如何自己执行它们。

&emsp;&emsp;使用闭源模型的过程与我们目前看到的开源示例有很大的不同。我们不需要加载模型，而是可以通过OpenAI的API访问模型。

&emsp;&emsp;在我们进入分类示例之前，首先需要在https://oreil.ly/AEXvA 上创建一个免费帐户，并在这里创建一个API密钥:https://oreil.ly/lrTXl 。这样做之后，您可以使用您的API与OpenAI的服务器进行通信。

&emsp;&emsp;我们可以用这个键来创建一个客户端:



In [ ]:
import openai

# Create client
client = openai.OpenAI(api_key="YOUR_KEY_HERE")

&emsp;&emsp;使用该客户端，我们创建 chatgpt_generation 函数，它允许我们根据特定的提示、输入文档和选定的模型生成一些文本:

In [ ]:
def chatgpt_generation(prompt, document, model="gpt-3.5-turbo-0125"):
    """Generate an output based on a prompt and an input document."""
    messages=[
        {
            "role": "system",
            "content": "You are a helpful assistant."
            },
        {
            "role": "user",
            "content":   prompt.replace("[DOCUMENT]", document)
            }
    ]
    chat_completion = client.chat.completions.create(
      messages=messages,
      model=model,
      temperature=0
    )
    return chat_completion.choices[0].message.content

&emsp;&emsp;接下来，我们需要创建一个模板来要求模型执行分类:

In [ ]:
# Define a prompt template as a base
prompt = """Predict whether the following document is a positive or negative movie review:

[DOCUMENT]

If it is positive return 1 and if it is negative return 0. Do not give any other answers.
"""

# Predict the target using GPT
document = "unpretentious , charming , quirky , original"
chatgpt_generation(prompt, document)

&emsp;&emsp;这个模板只是一个例子，你可以随意修改。现在，我们尽可能简单地说明如何使用这样的模板。

&emsp;&emsp;在你在一个潜在的大数据集上使用它之前，重要的是要始终跟踪你的使用情况。如果执行许多请求，OpenAI等外部api可能很快就会变得昂贵。在撰写本文时，使用“gpt-3.5-turbo-0125”模型运行我们的测试数据集的成本为3美分，该费用由免费帐户支付，但这在未来可能会发生变化。

&emsp;&emsp;接下来，我们可以对测试数据集中的所有评论运行此操作，以获得其预测结果。

In [ ]:
# You can skip this if you want to save your (free) credits
predictions = [
    chatgpt_generation(prompt, doc) for doc in tqdm(data["test"]["text"])
]

&emsp;&emsp;和前面的例子一样，我们需要将输出从字符串转换为整数来评估它的性能:

In [ ]:
# Extract predictions
y_pred = [int(pred) for pred in predictions]

# Evaluate performance
evaluate_performance(data["test"]["label"], y_pred)

&emsp;&emsp;运行结果为：


In [ ]:
***                precision   recall   f1-score   support

Negative Review       0.87      0.97      0.92       533
Positive Review       0.96      0.86      0.91       533

       accuracy                           0.91      1066
      macro avg       0.92      0.91      0.91      1066
   weighted avg       0.92      0.91      0.91      1066

&emsp;&emsp;F1分数0.91已经让我们看到了这个将生成式AI带给大众的模型的表现。然而，由于我们不知道模型是在什么数据上进行训练的，所以我们不能轻易地使用这类指标来评估模型。就我们所知，它可能实际上是在我们的数据集上训练的!

&emsp;&emsp;在第12章中，我们将探索如何在更一般化的任务上评估开源和闭源模型。

# 总结

&emsp;&emsp;在本章中，我们讨论了许多不同的技术来执行各种各样的分类任务，从微调整个模型到根本不调优!对文本数据进行分类并不像表面上看起来那么简单，而且有大量的创造性技术可以做到这一点。

&emsp;&emsp;在本章中，我们探索了使用生成语言模型和表示语言模型的文本分类。我们的目标是分配一个标签或类来输入文本，用于对评论的情感进行分类。

&emsp;&emsp;我们探索了两种类型的表示模型，一种是任务特定模型，另一种是嵌入模型。特定任务模型在一个专门用于情感分析的大型数据集上进行了预训练，并向我们展示了预训练模型是一种很好的文档分类技术。嵌入模型用于生成多用途嵌入，我们将其用作训练分类器的输入。

&emsp;&emsp;同样，我们探索了两种类型的生成模型，一种是开源编码器-解码器模型(Flan-T5)，另一种是闭源解码器模型(GPT-3.5)。我们在文本分类中使用了这些生成模型，而不需要对领域数据或标记数据集进行特定的(额外的)训练。

&emsp;&emsp;在下一章中，我们将继续讨论分类，但将重点放在无监督分类上。如果我们有没有任何标签的文本数据，我们能做什么?我们可以提取哪些信息?我们将重点关注数据的聚类，以及使用主题建模技术对聚类进行命名。